<a href="https://colab.research.google.com/github/economicactivist/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### extract column names from text

import re
text = """
1. Class Name: 2 (democrat, republican)
2. handicapped-infants: 2 (y,n)
3. water-project-cost-sharing: 2 (y,n)
4. adoption-of-the-budget-resolution: 2 (y,n)
5. physician-fee-freeze: 2 (y,n)
6. el-salvador-aid: 2 (y,n)
7. religious-groups-in-schools: 2 (y,n)
8. anti-satellite-test-ban: 2 (y,n)
9. aid-to-nicaraguan-contras: 2 (y,n)
10. mx-missile: 2 (y,n)
11. immigration: 2 (y,n)
12. synfuels-corporation-cutback: 2 (y,n)
13. education-spending: 2 (y,n)
14. superfund-right-to-sue: 2 (y,n)
15. crime: 2 (y,n)
16. duty-free-exports: 2 (y,n)
17. export-administration-act-south-africa: 2 (y,n)
"""
pattern = re.compile(r'\b[a-zA-Z -]+(?=:)')

col_names = re.findall(pattern=pattern, string=text)



In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data', names=col_names)

In [105]:
df.head()

,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [106]:
df.columns.str.replace("-", "_").str.replace("Class Name", "party")

Index(['party', 'handicapped_infants', 'water_project_cost_sharing',
       'adoption_of_the_budget_resolution', 'physician_fee_freeze',
       'el_salvador_aid', 'religious_groups_in_schools',
       'anti_satellite_test_ban', 'aid_to_nicaraguan_contras', 'mx_missile',
       'immigration', 'synfuels_corporation_cutback', 'education_spending',
       'superfund_right_to_sue', 'crime', 'duty_free_exports',
       'export_administration_act_south_africa'],
      dtype='object')

In [0]:
df.columns = df.columns.str.replace("-", "_").str.replace("Class Name", "party")

In [108]:
df.head()

,party,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporation_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


###Null hypothesis:  There is no significant difference in the voting tendencies of republicans versus democrats

In [109]:
df.loc[2, 'handicapped_infants'] #check for spaces

'?'

In [110]:
df.info()  
"""if there are other forms of indicating null, this total non-null values 
shouldn't change after replacement"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 17 columns):
party                                     435 non-null object
handicapped_infants                       435 non-null object
water_project_cost_sharing                435 non-null object
adoption_of_the_budget_resolution         435 non-null object
physician_fee_freeze                      435 non-null object
el_salvador_aid                           435 non-null object
religious_groups_in_schools               435 non-null object
anti_satellite_test_ban                   435 non-null object
aid_to_nicaraguan_contras                 435 non-null object
mx_missile                                435 non-null object
immigration                               435 non-null object
synfuels_corporation_cutback              435 non-null object
education_spending                        435 non-null object
superfund_right_to_sue                    435 non-null object
crime                      

"if there are other forms of indicating null, this total non-null values \nshouldn't change after replacement"

In [0]:
df = df.replace("?", np.nan)

In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 17 columns):
party                                     435 non-null object
handicapped_infants                       423 non-null object
water_project_cost_sharing                387 non-null object
adoption_of_the_budget_resolution         424 non-null object
physician_fee_freeze                      424 non-null object
el_salvador_aid                           420 non-null object
religious_groups_in_schools               424 non-null object
anti_satellite_test_ban                   421 non-null object
aid_to_nicaraguan_contras                 420 non-null object
mx_missile                                413 non-null object
immigration                               428 non-null object
synfuels_corporation_cutback              414 non-null object
education_spending                        404 non-null object
superfund_right_to_sue                    410 non-null object
crime                      

In [113]:
df.export_administration_act_south_africa.isnull().mean()

0.23908045977011494

~24% of the "south africa" column is null.  Possibly should be dropped

In [0]:
repub = df[df.party=='republican']

In [0]:
democ = df[df.party=='democrat']

In [116]:
repub.replace({'y': 1, 'n':0})

,party,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporation_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,republican,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
430,republican,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
432,republican,0.0,NaN,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
433,republican,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,0.0,1.0,1.0,1.0,0.0,1.0


In [0]:
repub_encode = repub.replace({'y': 1, 'n':0})  
#encode binary categories to simplify calculations

In [0]:
democ_encode = democ.replace({'y': 1, 'n':0})
#encode binary categories to simplify calculations

###Check out all the means

In [119]:
democ_encode.describe().loc['mean', :]

handicapped_infants                       0.604651
water_project_cost_sharing                0.502092
adoption_of_the_budget_resolution         0.888462
physician_fee_freeze                      0.054054
el_salvador_aid                           0.215686
religious_groups_in_schools               0.476744
anti_satellite_test_ban                   0.772201
aid_to_nicaraguan_contras                 0.828897
mx_missile                                0.758065
immigration                               0.471483
synfuels_corporation_cutback              0.505882
education_spending                        0.144578
superfund_right_to_sue                    0.289683
crime                                     0.350195
duty_free_exports                         0.637450
export_administration_act_south_africa    0.935135
Name: mean, dtype: float64

In [120]:
repub_encode.describe().loc['mean', :]

handicapped_infants                       0.187879
water_project_cost_sharing                0.506757
adoption_of_the_budget_resolution         0.134146
physician_fee_freeze                      0.987879
el_salvador_aid                           0.951515
religious_groups_in_schools               0.897590
anti_satellite_test_ban                   0.240741
aid_to_nicaraguan_contras                 0.152866
mx_missile                                0.115152
immigration                               0.557576
synfuels_corporation_cutback              0.132075
education_spending                        0.870968
superfund_right_to_sue                    0.860759
crime                                     0.981366
duty_free_exports                         0.089744
export_administration_act_south_africa    0.657534
Name: mean, dtype: float64

In [0]:
from scipy.stats import ttest_ind

##Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01

In [122]:
ttest_ind(democ_encode.adoption_of_the_budget_resolution.dropna(), repub_encode.adoption_of_the_budget_resolution.dropna())

Ttest_indResult(statistic=23.21277691701378, pvalue=2.0703402795405602e-77)

In [123]:
len(democ_encode.adoption_of_the_budget_resolution.dropna())

260

In [124]:
len(repub_encode.adoption_of_the_budget_resolution.dropna()) #lower length chosen

164

Due to observing a t-statistic of **23.21277** and a resulting p value of  **2.07 * 10^-77** (basically zero) 
we reject the null hypothesis that there is no significant difference in voting between the parties in reference to the adoption of the budject resolution and suggest the alternative hypothesis that there is a significant difference. 







---------------------------------

## Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01

In [125]:
ttest_ind(democ_encode.physician_fee_freeze.dropna(), repub_encode.physician_fee_freeze.dropna())

Ttest_indResult(statistic=-49.367081573014055, pvalue=1.994262314074572e-177)

In [126]:
len(repub_encode.physician_fee_freeze.dropna()) #lower length chosen

165

In [127]:
len(democ_encode.physician_fee_freeze.dropna())

259

Due to observing a t-statistic of **-49.367081573014055** and a resulting p value of  **1.99 * 10^-177** (basically zero) 
we reject the null hypothesis that there is no significant difference in voting between the parties in reference to the physician fee freeze and suggest the alternative hypothesis that there is a significant difference. 











-----------------------
## Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference) 

In [135]:
ttest_ind(democ_encode.water_project_cost_sharing.dropna(), repub_encode.water_project_cost_sharing.dropna())

Ttest_indResult(statistic=-0.08896538137868286, pvalue=0.9291556823994811)

In [136]:
len(repub_encode.water_project_cost_sharing.dropna()) #lower length chosen

148

In [137]:
len(democ_encode.water_project_cost_sharing.dropna())

239

Due to observing a t-statistic of -0.08896538137868286 and a resulting p value of 0.9291556823994811 we fail to reject the null hypothesis that there is no significant difference in voting between the parties in reference to the water project cost sharing agenda.

Notes:

In [0]:
#low t-test statistic, high p-value

In [0]:
#confidence level: how willing am I to be tricked by randomness?

In [0]:
#p value ~ prob that null is true

In [131]:
"""Due to observing a t-statistic of_________ and a resulting p value of  ________ 
we reject the null hypothesis that __________ and suggest the alternative hypothesis
that they are different"""

'Due to observing a t-statistic of_________ and a resulting p value of  ________ \nwe reject the null hypothesis that __________ and suggest the alternative hypothesis\nthat they are different'